## File search assistant
The McKessonMDTranscripts demonstrates the creation and execution of an assistant using the file_search feature with a knowledge base based on earnings call transcripts downloaded from the internet.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.7.7"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.7.7

## Ensure the McKessonMDTranscripts assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\McKessonMDTranscripts" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"McKessonMDTranscripts" contains the following files:
```
│   manifest.json
│   prompt.md
│
└───VectorStores
    └───McKessonMDTranscripts
            211101-MCK-Q2FY22-Earnings-Call-Transcript.md
            220202-MCK-Q3FY22-Earnings-Call-Transcript.md
            220505-MCK-Q4FY22-Earnings-Call-Transcript.md
            220803-MCK-Q1FY23-Earnings-Call-Transcript.md
            230508-MCK-Q4FY23-Earnings-Call-Transcript.md
            230802-MCK-Q1FY24-Earnings-Call-Transcript.md
            231101-MCK-Q2FY24-Earnings-Call-Transcript.md
            MCK-Q1FY21-Transcript.md
            MCK-Q1FY22-Transcript.md
            MCK-Q2-FY23-Earnings-Transcript.md
            MCK-Q2FY21-Transcript.md
            MCK-Q3-FY24-Earnings-Transcript.md
            MCK-Q4-FY24-Earnings-Call-Transcript.md
            MCK-Q4FY21-Transcript.md
            MCK-US-20230201-2761065-C.md
            Q1-FY19-Earnings-Call-Transcript.md
            Q1-FY20-Earnings-Call-Transcript.md
            Q2-FY19-Earnings-Call-Transcript.md
            Q2-FY20-Earnings-Call-Transcript.md
            Q3-FY19-Earnings-Call-Transcript.md
            Q3FY20-MCK-Earnings-Call-Transcript.md
            Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md
            Q4-FY19-Earnings-Call-Transcript.md
            Q4FY20-MCK-Corrected-Transcript.md
```

### Explanation
`AssistantUtility.Create` will create the vector store if necessary, upload the files, and then update the manifest with the vector store ID before creating the assistant.

In [2]:
var assistantId = await AssistantUtility.Create("McKessonMDTranscripts", config);
Console.WriteLine(assistantId)

asst_SqufMGVEUynBcBCb6vSlPJdB


## Run the Assistant

`output.Dialog` shows the conversation.

In [3]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "McKessonMDTranscripts",
    Instructions = "How did covid impact results over time?",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: How did covid impact results over time?

Assistant: I searched my knowledge base for the answer.

COVID-19 had significant and varied impacts on McKesson's financial results over time. Here is a summary of the key impacts:

1. **Fiscal 2020**:
   - The onset of the COVID-19 pandemic led to increased demand for pharmaceuticals and personal protective equipment (PPE), resulting in approximately $2 billion of incremental revenue in the fourth quarter【4:0†source】【4:3†source】.
   - Adjusted gross profit for the fourth quarter was positively impacted by $65 million due to these increased volumes, although there were also higher operating expenses related to supporting frontline employees and a remote workforce【4:3†source】.
   - The increased demand and consumer behavior changes provided a net tailwind to the fourth quarter revenue results【4:3†source】.

2. **Fiscal 2021**:
   - The first half of fiscal 2021 was challenging due to the pandemic, with expectations of gradual improvement in the second half【4:0†source】【4:4†source】.
   - McKesson experienced increased demand for COVID-19 tests and uneven levels of demand for PPE throughout the fiscal year【4:1†source】.
   - The company played a significant role in distributing COVID-19 vaccines and ancillary supply kits, which contributed to the financial results in the U.S. Pharmaceutical and Medical-Surgical segments【4:4†source】【4:17†source】.
   - There was also a benefit from unplanned gains on equity investments within the McKesson Ventures portfolio【4:11†source】.
   - Despite the challenges, McKesson's core business remained fundamentally sound, and the company increased its fiscal year 2021 guidance due to the execution of vaccine and kitting programs【4:12†source】【4:15†source】.

3. **Fiscal 2022**:
   - The demand for COVID-19-related products fluctuated, with a spike in COVID-19 tests in January 2022 due to the Omicron variant, followed by a decline in February and March【4:6†source】.
   - The company recognized lower contributions from the U.S. government's COVID-19 programs, but these were partially offset by commercial COVID-19 distribution【4:14†source】.
   - For the full year, COVID-19-related items provided a benefit of $216 million, compared to $371 million in fiscal 2022【4:2†source】.

4. **Fiscal 2023**:
   - Lower sales of COVID-19 tests and lower contributions from kitting, storage, and distribution of ancillary supplies to the U.S. government's COVID-19 program impacted the financial results【4:8†source】.
   - The company's International segment saw a decrease in revenues and operating profit due to the completed divestitures within European businesses【4:8†source】.
   - The contract with the U.S. government for COVID-19 vaccine distribution provided a benefit of $149 million for the full year, down from $186 million in fiscal 2022【4:8†source】.

Overall, the impact of COVID-19 on McKesson's results varied over time, with initial increases in demand for pharmaceuticals and PPE, followed by contributions from vaccine distribution, and later fluctuations in demand for COVID-19-related products and services.


In [4]:
output.LastMessage.DisplayAs("text/markdown");

COVID-19 had significant and varied impacts on McKesson's financial results over time. Here is a summary of the key impacts:

1. **Fiscal 2020**:
   - The onset of the COVID-19 pandemic led to increased demand for pharmaceuticals and personal protective equipment (PPE), resulting in approximately $2 billion of incremental revenue in the fourth quarter【4:0†source】【4:3†source】.
   - Adjusted gross profit for the fourth quarter was positively impacted by $65 million due to these increased volumes, although there were also higher operating expenses related to supporting frontline employees and a remote workforce【4:3†source】.
   - The increased demand and consumer behavior changes provided a net tailwind to the fourth quarter revenue results【4:3†source】.

2. **Fiscal 2021**:
   - The first half of fiscal 2021 was challenging due to the pandemic, with expectations of gradual improvement in the second half【4:0†source】【4:4†source】.
   - McKesson experienced increased demand for COVID-19 tests and uneven levels of demand for PPE throughout the fiscal year【4:1†source】.
   - The company played a significant role in distributing COVID-19 vaccines and ancillary supply kits, which contributed to the financial results in the U.S. Pharmaceutical and Medical-Surgical segments【4:4†source】【4:17†source】.
   - There was also a benefit from unplanned gains on equity investments within the McKesson Ventures portfolio【4:11†source】.
   - Despite the challenges, McKesson's core business remained fundamentally sound, and the company increased its fiscal year 2021 guidance due to the execution of vaccine and kitting programs【4:12†source】【4:15†source】.

3. **Fiscal 2022**:
   - The demand for COVID-19-related products fluctuated, with a spike in COVID-19 tests in January 2022 due to the Omicron variant, followed by a decline in February and March【4:6†source】.
   - The company recognized lower contributions from the U.S. government's COVID-19 programs, but these were partially offset by commercial COVID-19 distribution【4:14†source】.
   - For the full year, COVID-19-related items provided a benefit of $216 million, compared to $371 million in fiscal 2022【4:2†source】.

4. **Fiscal 2023**:
   - Lower sales of COVID-19 tests and lower contributions from kitting, storage, and distribution of ancillary supplies to the U.S. government's COVID-19 program impacted the financial results【4:8†source】.
   - The company's International segment saw a decrease in revenues and operating profit due to the completed divestitures within European businesses【4:8†source】.
   - The contract with the U.S. government for COVID-19 vaccine distribution provided a benefit of $149 million for the full year, down from $186 million in fiscal 2022【4:8†source】.

Overall, the impact of COVID-19 on McKesson's results varied over time, with initial increases in demand for pharmaceuticals and PPE, followed by contributions from vaccine distribution, and later fluctuations in demand for COVID-19-related products and services.

## Annotations

In [5]:
Console.WriteLine(JsonSerializer.Serialize(output.Annotations, new JsonSerializerOptions() { WriteIndented = true }));

[
  {
    "type": "file_citation",
    "text": "\u30104:0\u2020source\u3011",
    "start_index": 353,
    "end_index": 365,
    "file_citation": {
      "file_id": "assistant-v1vgOsajg7zOoaHPJaHKvA3i",
      "file_name": "",
      "quote": null
    }
  },
  {
    "type": "file_citation",
    "text": "\u30104:3\u2020source\u3011",
    "start_index": 365,
    "end_index": 377,
    "file_citation": {
      "file_id": "assistant-v1vgOsajg7zOoaHPJaHKvA3i",
      "file_name": "",
      "quote": null
    }
  },
  {
    "type": "file_citation",
    "text": "\u30104:3\u2020source\u3011",
    "start_index": 615,
    "end_index": 627,
    "file_citation": {
      "file_id": "assistant-v1vgOsajg7zOoaHPJaHKvA3i",
      "file_name": "",
      "quote": null
    }
  },
  {
    "type": "file_citation",
    "text": "\u30104:3\u2020source\u3011",
    "start_index": 746,
    "end_index": 758,
    "file_citation": {
      "file_id": "assistant-v1vgOsajg7zOoaHPJaHKvA3i",
      "file_name": "",
      "quote"

### Formatting Output
The current version of the API leaves some things to be desired. First, because of chunking it can cite the same source twice. Second, 'quote' is supposed to contain a snippet, but currently, it is always null

In [6]:
var files = new Dictionary<string, string>();
var annotatedMessage = output.LastMessage;
foreach(var annotation in output.Annotations)
{
    // Only get the file name once per id
    if(!files.ContainsKey(annotation.FileCitation.FileId))
    {
        files[annotation.FileCitation.FileId] = await annotation.GetFileName(config);
    }
    // But the annotation itself may be a duplicate citation to the same file, so one must process them all
    // You can format the replacement however you like. In this case it just makes an md link
    annotatedMessage = annotatedMessage.Replace(annotation.Text, $" [{files[annotation.FileCitation.FileId]}]({files[annotation.FileCitation.FileId]})");
}

foreach(var file in files)
{
    Console.WriteLine($"{file.Key} = {file.Value}");
}
Console.WriteLine("-------------------");
annotatedMessage.DisplayAs("text/markdown");

assistant-v1vgOsajg7zOoaHPJaHKvA3i = McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md
assistant-mLYhwg7I9ywfWQPh1xEf1IXY = McKessonMDTranscripts_MCK-Q4FY21-Transcript.md
assistant-LtpQtNw0yqJbx07lbEJahob9 = McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md
assistant-KVLQgIZo6R8UzDv01PXDkrR6 = McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md
assistant-zRvCwWZqpjdqkhSsF9aC2hVN = McKessonMDTranscripts_MCK-Q3-FY24-Earnings-Transcript.md
assistant-fslYRzYxsqiFwPGd7Fns83se = McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md
-------------------


COVID-19 had significant and varied impacts on McKesson's financial results over time. Here is a summary of the key impacts:

1. **Fiscal 2020**:
   - The onset of the COVID-19 pandemic led to increased demand for pharmaceuticals and personal protective equipment (PPE), resulting in approximately $2 billion of incremental revenue in the fourth quarter [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md) [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md).
   - Adjusted gross profit for the fourth quarter was positively impacted by $65 million due to these increased volumes, although there were also higher operating expenses related to supporting frontline employees and a remote workforce [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md).
   - The increased demand and consumer behavior changes provided a net tailwind to the fourth quarter revenue results [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md).

2. **Fiscal 2021**:
   - The first half of fiscal 2021 was challenging due to the pandemic, with expectations of gradual improvement in the second half [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md) [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md).
   - McKesson experienced increased demand for COVID-19 tests and uneven levels of demand for PPE throughout the fiscal year [McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md](McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md).
   - The company played a significant role in distributing COVID-19 vaccines and ancillary supply kits, which contributed to the financial results in the U.S. Pharmaceutical and Medical-Surgical segments [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md) [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md).
   - There was also a benefit from unplanned gains on equity investments within the McKesson Ventures portfolio [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md).
   - Despite the challenges, McKesson's core business remained fundamentally sound, and the company increased its fiscal year 2021 guidance due to the execution of vaccine and kitting programs [McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md](McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md) [McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md](McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md).

3. **Fiscal 2022**:
   - The demand for COVID-19-related products fluctuated, with a spike in COVID-19 tests in January 2022 due to the Omicron variant, followed by a decline in February and March [McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md](McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md).
   - The company recognized lower contributions from the U.S. government's COVID-19 programs, but these were partially offset by commercial COVID-19 distribution [McKessonMDTranscripts_MCK-Q3-FY24-Earnings-Transcript.md](McKessonMDTranscripts_MCK-Q3-FY24-Earnings-Transcript.md).
   - For the full year, COVID-19-related items provided a benefit of $216 million, compared to $371 million in fiscal 2022 [McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md](McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md).

4. **Fiscal 2023**:
   - Lower sales of COVID-19 tests and lower contributions from kitting, storage, and distribution of ancillary supplies to the U.S. government's COVID-19 program impacted the financial results [McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md](McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md).
   - The company's International segment saw a decrease in revenues and operating profit due to the completed divestitures within European businesses [McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md](McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md).
   - The contract with the U.S. government for COVID-19 vaccine distribution provided a benefit of $149 million for the full year, down from $186 million in fiscal 2022 [McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md](McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md).

Overall, the impact of COVID-19 on McKesson's results varied over time, with initial increases in demand for pharmaceuticals and PPE, followed by contributions from vaccine distribution, and later fluctuations in demand for COVID-19-related products and services.

## Clean Up

In [7]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "McKessonMDTranscripts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
